In [ ]:
import numpy as np
from datetime import datetime

np.random.seed(0)

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = float(tmp_val) - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [ ]:
class Diabetes:
    def __init__(self, i_node, h_node, o_node, lr):
        self.learning_rate = lr
        
        self.W2 = np.random.rand(i_node, h_node)
        self.b2 = np.random.rand(1)
        
        self.W3 = np.random.rand(h_node, o_node)
        self.b3 = np.random.rand(1)
        
        print("========== Diabetes Object is created. ==========\n")
        
    def feed_forward(self, x_data, t_data):
        delta = 1e-7
        
        z2 = np.dot(x_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        return -np.sum((t_data*np.log10(a3 + delta)) + ((1 - t_data)*np.log10((1 - a3) + delta)))

    def loss_val(self, x_data, t_data):
        delta = 1e-7
        
        z2 = np.dot(x_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        return -np.sum((t_data*np.log10(a3 + delta)) + ((1 - t_data)*np.log10((1 - a3) + delta)))
    
    def train(self, x_data, t_data):
        f = lambda x : self.feed_forward(x_data, t_data)
        
        self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
        self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
        self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)
        
    def predict(self, test_data):
        z2 = np.dot(test_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        a3 = sigmoid(z3)
        
        if a3 >= 0.5:
            result = 1
        else:
            result = 0
    
        return a3, result
    
    def accuracy(self, test_xdata, test_tdata):
        matched_list = []
        not_matched_list = []
        
        for index in range(len(test_xdata)):
            (real_val, logical_val) = self.predict(test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_val = len(matched_list) / len(test_xdata)
        
        return accuracy_val

In [ ]:
try:
    training_data = np.loadtxt('./diabetes.csv', delimiter=',', dtype=np.float32)
except Exception as err:
    print(str(err))

In [ ]:
i_node = 8
h_node = 10
o_node = 1
lr = 1e-2
epoch = 20

obj1 = Diabetes(i_node, h_node, o_node, lr)

print("Neural Network Learning using Numerical Derivative...\n")

start_time = datetime.now()

for step in range(epoch):
    for index in range(len(training_data)):
        input_data = training_data[index, 0:-1]
        target_data = training_data[index, [-1]]
        
        obj1.train(input_data, target_data)
        
    if step % (int)(0.05*epoch) == 0:
        print("epochs = ", step, " : ", "loss value = ", obj1.loss_val(input_data, target_data))

end_time = datetime.now()
        
print("")
print("Elapsed Time => ", end_time - start_time)

In [ ]:
try:
    # test data
    test_data = np.loadtxt('./diabetes.csv', delimiter=',', dtype=np.float32)
    print("test_data.shape = ", test_data.shape)

    input_data = test_data[:, 0:-1]
    target_data = test_data[:, [-1]]
    
    accuracy_ret = obj1.accuracy(input_data, target_data)
    
    print('Accuracy => ', accuracy_ret)
    
except Exception as err:
    print(str(err))